##  Run AutoGen with LLM loaded on Local GPU (Nvidia RTX 3060 6GB)

In [13]:
llm_config = {"config_list" : [
    {
        # Let's choose the Meta's Llama 3.2 model with 3b parameters
        "model": "llama3.2:3b",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        "client_host": "http://127.0.0.1:11434",
    }
]}

In [14]:
from autogen import ConversableAgent


ash_agent = ConversableAgent(name="Ash", system_message="you are a comedian in a show between 2 comedians", llm_config=llm_config, human_input_mode="NEVER",
                              is_termination_msg= lambda msg: "TERMINATE" in msg["content"] )
john_agent = ConversableAgent(name="John", system_message="you are a comedian in a show between 2 comedians", llm_config=llm_config, human_input_mode="NEVER")

In [18]:
result = ash_agent.initiate_chat(john_agent, message={"content": "hello, could you tell me a joke? " }, max_turns=10)
result

Ash (to John):

hello, could you tell me a joke? 

--------------------------------------------------------------------------------
John (to Ash):

(laughs) Ah, hello there! Well, I'd love to make you laugh. Here's one for you:

"You know what's weird about marriage? It's like a game of Jenga. You start with a solid foundation, and then you take turns removing pieces until it all comes crashing down. But the difference is, with Jenga, you can just put the pieces back and try again... whereas in marriage, you're just like, 'Honey, I think we lost the ring'!" (laughs)

--------------------------------------------------------------------------------
Ash (to John):

(laughs) Oh man, that's great! You know what they say, "marriage is like a game of Jenga"... but I think you might have just taken it to the next level with that last line. (chuckles) I mean, losing the ring? That's like, the ultimate Jenga fail.

You know, I've got one for you:

"You ever notice how everyone's a foodie nowaday

ChatResult(chat_id=None, chat_history=[{'content': 'hello, could you tell me a joke? ', 'role': 'assistant', 'name': 'Ash'}, {'content': '(laughs) Ah, hello there! Well, I\'d love to make you laugh. Here\'s one for you:\n\n"You know what\'s weird about marriage? It\'s like a game of Jenga. You start with a solid foundation, and then you take turns removing pieces until it all comes crashing down. But the difference is, with Jenga, you can just put the pieces back and try again... whereas in marriage, you\'re just like, \'Honey, I think we lost the ring\'!" (laughs)', 'role': 'user', 'name': 'John'}, {'content': '(laughs) Oh man, that\'s great! You know what they say, "marriage is like a game of Jenga"... but I think you might have just taken it to the next level with that last line. (chuckles) I mean, losing the ring? That\'s like, the ultimate Jenga fail.\n\nYou know, I\'ve got one for you:\n\n"You ever notice how everyone\'s a foodie nowadays? \'Oh, I\'m a foodie.\' No, you\'re not a

### Code Executor
We can use code executor locally or in Docker, the safest is docker but for now we will use local executor

In [10]:
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
import os

os.makedirs(name="coding", exist_ok=True)
local_code_executor = LocalCommandLineCodeExecutor(timeout=10, work_dir="coding")

code_agent = ConversableAgent(name="code_agent",
                               code_execution_config={"executor": local_code_executor},
                               human_input_mode="ALWAYS", llm_config=False)





Ok now we can use this code executor to run python code that is going to draw a plot and save it as png image

In [11]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

In [12]:
reply = code_agent.generate_reply(messages=[{"role":"user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



In [14]:
# code exexutor with in a conversation 
# I need to build 2 agents the first is to write the code with specific detailed prompt, the other agent 
# is to execute the code and reply with the results. 



system_message_writer_agent = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done."""

llm_config = {"config_list" : [
    {
        # Let's choose the Meta's Llama 3.2 model with 3b parameters
        "model": "llama3.2:3b",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        "client_host": "http://127.0.0.1:11434",
    }
]}

code_writer_agent = ConversableAgent("code_writer_agent", system_message=system_message_writer_agent,
                                llm_config=llm_config,
                                code_execution_config=False)



In [19]:
chat_result = code_agent.initiate_chat(code_writer_agent, message="Write Python code to calculate e^14 and then write the result in a text file to disk. ")
chat_result


code_agent (to code_writer_agent):

Write Python code to calculate e^14 and then write the result in a text file to disk. 

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_agent):

```python
# filename: calc_e_to_power_14.py

import math

def calculate_e_to_power_14():
    # Calculate e^14 using math library
    result = math.exp(14)
    
    return result

result = calculate_e_to_power_14()
print("The calculated value of e^14 is:", result)

# Write the result to a text file named 'e_to_power_14.txt'
with open('e_to_power_14.txt', 'w') as file:
    file.write(str(result))
```

To execute this script, save it in a file (e.g., `calc_e_to_power_14.py`), and run the Python interpreter on it using the following command:

```bash
python calc_e_to_power_14.py
```

After executing this code, you should see the calculated value of e^14 printed to the console. Additionally, the result will be written to a f

ChatResult(chat_id=None, chat_history=[{'content': 'Write Python code to calculate e^14 and then write the result in a text file to disk. ', 'role': 'assistant', 'name': 'code_agent'}, {'content': '```python\n# filename: calc_e_to_power_14.py\n\nimport math\n\ndef calculate_e_to_power_14():\n    # Calculate e^14 using math library\n    result = math.exp(14)\n    \n    return result\n\nresult = calculate_e_to_power_14()\nprint("The calculated value of e^14 is:", result)\n\n# Write the result to a text file named \'e_to_power_14.txt\'\nwith open(\'e_to_power_14.txt\', \'w\') as file:\n    file.write(str(result))\n```\n\nTo execute this script, save it in a file (e.g., `calc_e_to_power_14.py`), and run the Python interpreter on it using the following command:\n\n```bash\npython calc_e_to_power_14.py\n```\n\nAfter executing this code, you should see the calculated value of e^14 printed to the console. Additionally, the result will be written to a file named `e_to_power_14.txt` in the same 